In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [2]:
pd.options.display.max_columns = 47

In [3]:
data = pd.read_csv('US_Accidents_Dec20_Updated.csv')

In [82]:
def accidents_ETL(accidents):
    
    #drop unnecessary columns
    dropthese = ['ID', 'End_Lat', 'End_Lng', 'End_Time',    'Description', 'Distance(mi)', 'Number', 'Street', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Start_Lat', 'Start_Lng', 'Nautical_Twilight', 'Astronomical_Twilight', 'Wind_Chill(F)', 'Amenity', 'Sunrise_Sunset']
    accidents = accidents.drop(columns = dropthese, axis = 1)

    #Remname Civil Twilight
    accidents = accidents.rename(columns={'Civil_Twilight': 'Day/Night'})

    #modify Start_Time to datetime so we can extract year, month, day, hour
    accidents['Start_Time'] = pd.to_datetime(accidents['Start_Time'])

    #extracting year, month, day, hour
    accidents['Year'] = pd.DatetimeIndex(accidents['Start_Time']).year
    accidents['Month'] = pd.DatetimeIndex(accidents['Start_Time']).month
    accidents['Day'] = pd.DatetimeIndex(accidents['Start_Time']).day
    accidents['Hour'] = pd.DatetimeIndex(accidents['Start_Time']).hour
    
    #drop unnecessary Start_Time column
    accidents = accidents.drop(['Start_Time'], axis=1)

    #remove expanded zip code value
    accidents2 = accidents['Zipcode'].str.split('-', expand=True)
    accidents['Zipcode'] = accidents2[0]

    #Combining City/State and County/State to prevent wrong city or county aggregation
    accidents['City_State'] = accidents['City'].astype(str) + ',' + accidents['State'].astype(str)
    accidents['County_State'] = accidents['County'].astype(str) + ',' + accidents['State'].astype(str)
    accidents['City'] = accidents['City_State']
    accidents['County'] = accidents['County_State']
    accidents = accidents.drop(['City_State', 'County_State'], axis=1)

    #Replacing wind directions to cardinal 8 directions
    accidents['Wind_Direction'].replace('CALM', 'Calm', inplace=True)
    accidents['Wind_Direction'].replace(['ENE', 'NNE'], 'NE', inplace=True)
    accidents['Wind_Direction'].replace(['ESE', 'SSE'], 'SE', inplace=True)
    accidents['Wind_Direction'].replace(['WNW', 'NNW'], 'NW', inplace=True)
    accidents['Wind_Direction'].replace(['WSW', 'SSW'], 'SW', inplace=True)
    accidents['Wind_Direction'].replace('North', 'N', inplace=True)
    accidents['Wind_Direction'].replace('East', 'E', inplace=True)
    accidents['Wind_Direction'].replace('South', 'S', inplace=True)
    accidents['Wind_Direction'].replace('West', 'W', inplace=True)
    accidents['Wind_Direction'].replace('VAR', 'Variable', inplace=True)

    #Chaning Weather_Condition NaN to "Unknown"
    accidents['Weather_Condition'] = accidents['Weather_Condition'].fillna('Unkown')

    #Chaning Temperature NaN to "Unknown"
    accidents['Temperature(F)'] = accidents['Temperature(F)'].fillna('Unkown')


    #Binning Humidity
    #Chaning Humidity NaN to "0"
    accidents['Humidity(%)'] = accidents['Humidity(%)'].fillna('0')
    #change column to numerical
    accidents['Humidity(%)'] = accidents['Humidity(%)'].astype(int)
    #Binning Humidity
    bins = [0, 1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    #create labels for the bins
    group_labels = ["Unknown", "0-9%", "10-19%", "20-29%", "30-39%", "40-49%", "50-59%", "60-69%", "70-79%", "80-89%", "90-100%"] 
    #slice the Humidity data and place into bins
    accidents['Humidity(%)'] = pd.cut(accidents['Humidity(%)'], bins, labels=group_labels)
    accidents


    #Chaning Pressure NaN to "Unknown"
    accidents['Pressure(in)'] = accidents['Pressure(in)'].fillna('Unkown')

    #Dropping Day/Night NaN values - because daylight hours change and cannot be reliably determined by time
    noDayNight = accidents[accidents['Day/Night'].isnull()]
    accidents = accidents.drop(noDayNight.index)


    #Filling in missing Zipcodes
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'St. Petersburg,FL'), '33713', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'District 4 Kent Island,MD'), '21666', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'Ross Valley,CA'), '94939', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'Springville-Mapleton,UT'), '84663', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'West Contra Costa,CA'), '94530', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'San Mateo,CA'), '94403', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'East Tehama,CA'), '96090', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'Southeast Marin,CA'), '94956', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'District 5,MD'), '20659', accidents.Zipcode)
    accidents['Zipcode'] = np.where((accidents['Zipcode'].isnull()) & (accidents['City'] == 'Avalon-Mulat,FL'), '32583', accidents.Zipcode)

    #Removing remaining NaN Zipcodes:
    no_zipcode = accidents[accidents['Zipcode'].isnull()]
    accidents = accidents.drop(no_zipcode.index)

    #Correcting Precipitation(in) NaN to 0 because if there was precipitation it would have likely been recorded. 
    accidents['Precipitation(in)'] = accidents['Precipitation(in)'].fillna(0.00)

    #Correcting Wind_Speed(mph) NaN to 0 because if there was wind it would have likely been recorded. 
    accidents['Wind_Speed(mph)'] = accidents['Wind_Speed(mph)'].fillna(0.0)

    #Changing unknown Wind_Direction with Wind_Speed(mi)>0 to "Variable"
    accidents['Wind_Direction'] = np.where((accidents['Wind_Direction'].isnull()) & (accidents['Wind_Speed(mph)'] > 0), 'Variable', accidents.Wind_Direction)

    #Changing Wind_Direction NaN with Wind_Speed(mi) of 0 to "Calm"
    accidents['Wind_Direction'] = np.where((accidents['Wind_Direction'].isnull()) & (accidents['Wind_Speed(mph)'] == 0), 'Calm', accidents.Wind_Direction)

    #Change Visibilty(mi) NaN values to the median of 10.0
    accidents['Visibility(mi)'] = accidents['Visibility(mi)'].fillna(10.00)


    #REMOVING OUTLIERS:

    #eliminating Wind_Speed outliers
    high_wind = accidents[accidents['Wind_Speed(mph)'] > 100]
    accidents = accidents.drop(high_wind.index)

    #eliminating Precipitation(in) outliers
    high_rain = accidents[accidents['Precipitation(in)'] > 13]
    accidents = accidents.drop(high_rain.index)

    #Weather Consolidtation
    #Clear
    accidents['Weather_Condition'].replace('Clear', 'Fair', inplace=True)
    accidents['Weather_Condition'].replace('N/A Precipitation', 'Fair', inplace=True)

    #Unknown
    accidents['Weather_Condition'].replace('N/A Preciptiation', 'Unknown', inplace=True)

    #Cloudy
    accidents['Weather_Condition'].replace('Overcast', 'Mostly Cloudy', inplace=True)
    accidents['Weather_Condition'].replace('Scattered Clouds', 'Partly Cloudy', inplace=True)

    #Rain
    accidents['Weather_Condition'].replace('Light Rain Shower', 'Light Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Rain Showers', 'Light Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Drizzle', 'Light Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Freezing Drizzle', 'Light Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Freezing Rain', 'Light Rain', inplace=True)
    accidents['Weather_Condition'].replace('Drizzle', 'Light Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Drizzle', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Showers in the Vicinity', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Rain Showers', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Rain Shower', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Freezing Rain', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Rain Shower', 'Heavy Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Freezing Rain', 'Heavy Rain', inplace=True)
    accidents['Weather_Condition'].replace('Freezing Drizzle', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Rain Showers', 'Heavy Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Freezing Drizzle', 'Rain', inplace=True)

    #Fog
    accidents['Weather_Condition'].replace('Patches of Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Light Freezing Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Partial Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Light Fog', 'Fog', inplace=True)

    #Smoke/Haze
    accidents['Weather_Condition'].replace('Smoke', 'Smoke/Haze', inplace=True)
    accidents['Weather_Condition'].replace('Haze', 'Smoke/Haze', inplace=True)
    accidents['Weather_Condition'].replace('Light Haze', 'Smoke/Haze', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Smoke', 'Smoke/Haze', inplace=True)

    #Thunderstorms
    accidents['Weather_Condition'].replace('T-Storm', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Light Thunderstorms and Rain', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Thunder in the Vicinity', 'Thunder', inplace=True)
    accidents['Weather_Condition'].replace('Light Rain with Thunder', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Thunderstorms and Rain', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Heavy T-Storm', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Thunderstorms and Rain', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Light Thunderstorms and Snow', 'Thunderstorms/Snow', inplace=True)
    accidents['Weather_Condition'].replace('Thunderstorms and Snow', 'Thunderstorms/Snow', inplace=True)
    accidents['Weather_Condition'].replace('Light Thunderstorm', 'Thunderstorm', inplace=True)

    #Snow
    accidents['Weather_Condition'].replace('Light Snow Showers', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Snow Grains', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Light Blowing Snow', 'Snow/Windy', inplace=True)
    accidents['Weather_Condition'].replace('Light Snow Grains', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Low Drifting Snow', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Snow Showers', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Blowing Snow', 'Heavy Snow/Windy', inplace=True)
    accidents['Weather_Condition'].replace('Light Snow Shower', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Drifting Snow', 'Snow/Windy', inplace=True)

    #Hail
    accidents['Weather_Condition'].replace('Light Ice Pellets', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Ice Pellets', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Small Hail', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Ice Pellets', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Light Hail', 'Hail', inplace=True)

    #Sleet
    accidents['Weather_Condition'].replace('Light Sleet', 'Sleet', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Sleet', 'Sleet', inplace=True)

    #Dust
    accidents['Weather_Condition'].replace('Sand', 'Dust', inplace=True)
    accidents['Weather_Condition'].replace('Widespread Dust', 'Dust', inplace=True)

    #Tornado
    accidents['Weather_Condition'].replace('Tornado', 'Funnel Cloud', inplace=True)                          



    #Converting Bool columns to integers to prepare for scaling and ML modeling
    bool_cols = accidents.dtypes[accidents.dtypes == 'bool'].index.tolist()
    bool_int = accidents[bool_cols].astype(int)
    encoding = accidents.copy()
    encoding = accidents.drop(bool_int,1)
    #encoding = encoding.merge(bool_int, left_index=True, right_index=True)
    accidents = encoding.merge(bool_int, left_index=True, right_index=True)


    #Encoding Obj columns using OneHotEncoder


    return(accidents)

In [83]:
accidents = accidents_ETL(data)
accidents

,Severity,Side,City,County,State,Zipcode,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Day/Night,Year,Month,Day,Hour,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,2,R,"Greenville,SC","Greenville,SC",SC,29607,76.0,50-59%,28.91,10.0,N,7.0,0.0,Fair,Day,2019,5,21,8,0,0,0,0,0,0,0,0,0,0,0,0
1,2,R,"Charlotte,NC","Mecklenburg,NC",NC,28270,76.0,60-69%,29.3,10.0,Variable,3.0,0.0,Cloudy,Day,2019,10,7,17,0,0,0,0,0,0,0,0,0,0,0,0
2,2,R,"Los Gatos,CA","Santa Clara,CA",CA,95033,51.0,70-79%,30.17,10.0,W,6.0,0.0,Fair,Night,2020,12,13,21,0,0,0,0,0,0,0,0,0,0,0,0
3,2,R,"Carson City,NV","Douglas,NV",NV,89705,53.6,10-19%,30.16,10.0,SW,4.6,0.0,Fair,Day,2018,4,17,16,0,0,0,0,0,0,0,0,0,0,1,0
4,3,R,"Fort Lauderdale,FL","Broward,FL",FL,33324,84.2,80-89%,29.92,10.0,SE,13.8,0.0,Mostly Cloudy,Day,2016,8,31,17,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906605,2,L,"Houston,TX","Harris,TX",TX,77018,84.2,60-69%,30.02,9.0,Variable,5.8,0.0,Fair,Day,2018,6,28,8,0,0,0,0,0,0,0,0,1,0,0,0
2906606,2,R,"Colton,CA","San Bernardino,CA",CA,92324,46.9,70-79%,30.14,10.0,Calm,0.0,0.0,Fair,Night,2019,1,10,2,0,0,0,0,0,0,0,0,0,0,0,0
2906607,2,L,"Miami,FL","Miami-Dade,FL",FL,33173,76.0,80-89%,30.0,10.0,NW,16.0,0.0,Mostly Cloudy,Day,2020,11,23,12,0,0,0,0,0,0,0,0,0,0,0,0
2906608,2,R,"Salt Lake City,UT","Salt Lake,UT",UT,84129,27.0,80-89%,25.81,10.0,SE,8.0,0.0,Cloudy,Night,2019,12,29,22,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
pd.options.display.max_rows = None
test = accidents['Humidity(%)']

desc = test.describe()
nan = test.isnull().sum()
counts = test.value_counts(dropna=False)

print(f'Stats: {desc}')
print(f'NaN: {nan}')
print(f'Value Counts: {counts}')
pd.options.display.max_rows = 20

Stats: count    2.906065e+06
mean     6.378267e+01
std      2.474636e+01
min      0.000000e+00
25%      4.700000e+01
50%      6.700000e+01
75%      8.400000e+01
max      1.000000e+02
Name: Humidity(%), dtype: float64
NaN: 0
Value Counts: 100    112856
93     112558
0       70900
90      64559
87      64275
89      57556
96      53117
86      50909
82      48304
81      47507
83      47341
94      46185
78      45958
84      45849
74      45098
72      44341
67      43471
65      42435
79      42018
76      41832
69      41020
77      40960
70      40933
63      40933
73      40133
61      39719
59      39480
75      38826
88      38604
68      38370
62      38205
80      37967
57      37466
66      37278
71      37218
52      37017
55      36982
60      36772
64      35997
58      35613
53      35262
54      34928
97      34534
56      33999
50      33644
48      33038
49      31675
51      31662
85      31035
46      30554
47      29492
45      28664
44      28550
43      27914
42    

In [22]:
obj_cols = accidents.dtypes[accidents.dtypes == 'object'].index.tolist()
obj_cols

['Side',
 'City',
 'County',
 'State',
 'Zipcode',
 'Temperature(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Wind_Direction',
 'Weather_Condition',
 'Day/Night']

In [12]:
enc = OneHotEncoder(sparse=False)

In [14]:
weather_encode_df = pd.DataFrame(enc.fit_transform(accidents[initial_cols].values.reshape(-1,1)))

ValueError: Boolean array expected for the condition, not object

In [ ]:
weather_encode_df.columns = enc.get_feature_names(obj_cols)
weather_encode_df.head()

In [76]:
pd.options.display.max_rows = None
print(accidents.dtypes)
pd.options.display.max_rows = 20

Severity                int64
Side                   object
City                   object
County                 object
State                  object
Zipcode                object
Temperature(F)         object
Humidity(%)             int64
Pressure(in)           object
Visibility(mi)        float64
Wind_Direction         object
Wind_Speed(mph)       float64
Precipitation(in)     float64
Weather_Condition      object
Day/Night              object
Year                    int64
Month                   int64
Day                     int64
Hour                    int64
Bump                    int64
Crossing                int64
Give_Way                int64
Junction                int64
No_Exit                 int64
Railway                 int64
Roundabout              int64
Station                 int64
Stop                    int64
Traffic_Calming         int64
Traffic_Signal          int64
Turning_Loop            int64
Humidity             category
dtype: object


In [72]:
accidents

,Severity,Side,City,County,State,Zipcode,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Day/Night,Year,Month,Day,Hour
0,2,R,"Greenville,SC","Greenville,SC",SC,29607,76,52,28.91,10.0,N,7.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,Day,2019,5,21,8
1,2,R,"Charlotte,NC","Mecklenburg,NC",NC,28270,76,62,29.3,10.0,Variable,3.0,0.0,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,Day,2019,10,7,17
2,2,R,"Los Gatos,CA","Santa Clara,CA",CA,95033,51,80,30.17,10.0,W,6.0,0.0,Fair,False,False,False,False,False,False,False,False,False,False,False,False,Night,2020,12,13,21
3,2,R,"Carson City,NV","Douglas,NV",NV,89705,53.6,16,30.16,10.0,SW,4.6,0.0,Clear,False,False,False,False,False,False,False,False,False,False,True,False,Day,2018,4,17,16
4,3,R,"Fort Lauderdale,FL","Broward,FL",FL,33324,84.2,84,29.92,10.0,SE,13.8,0.0,Overcast,False,False,False,True,False,False,False,False,False,False,True,False,Day,2016,8,31,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906605,2,L,"Houston,TX","Harris,TX",TX,77018,84.2,70,30.02,9.0,Variable,5.8,0.0,Clear,False,False,False,False,False,False,False,False,True,False,False,False,Day,2018,6,28,8
2906606,2,R,"Colton,CA","San Bernardino,CA",CA,92324,46.9,74,30.14,10.0,Calm,NaN,0.0,Clear,False,False,False,False,False,False,False,False,False,False,False,False,Night,2019,1,10,2
2906607,2,L,"Miami,FL","Miami-Dade,FL",FL,33173,76,85,30,10.0,NW,16.0,0.0,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,Day,2020,11,23,12
2906608,2,R,"Salt Lake City,UT","Salt Lake,UT",UT,84129,27,81,25.81,10.0,SE,8.0,0.0,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,Night,2019,12,29,22
